In [44]:
from itertools import combinations
import numpy as np
import random
from icecream import ic
from typing import List, NamedTuple

## Simple Test Problem

In [45]:
CITIES = [
    "Rome",
    "Milan",
    "Naples",
    "Turin",
    "Palermo",
    "Genoa",
    "Bologna",
    "Florence",
    "Bari",
    "Catania",
    "Venice",
    "Verona",
    "Messina",
    "Padua",
    "Trieste",
    "Taranto",
    "Brescia",
    "Prato",
    "Parma",
    "Modena",
]
test_problem = np.load("lab2/test_problem.npy")

## Common tests

In [46]:
problem = np.load("lab2/problem_r2_100.npy")

In [47]:
# Negative values?
np.any(problem < 0)

np.True_

In [48]:
# Diagonal is all zero?
np.allclose(np.diag(problem), 0.0)

False

In [49]:
# Symmetric matrix?
np.allclose(problem, problem.T)

False

In [50]:
# Triangular inequality
all(
    problem[x, y] <= problem[x, z] + problem[z, y]
    for x, y, z in list(combinations(range(problem.shape[0]), 3))
)

False

In [51]:
class Individual(NamedTuple):
    tour: List[int]
    fitness: float | None = None


def fitness(tour, dist: np.ndarray) -> float:
    """compute total tour fitness (closed loop)"""
    t = np.array(tour)
    return float(np.sum(dist[t, np.roll(t, -1)]))


def make_individual(solution: List[int], distance_matrix: np.ndarray) -> Individual:
    return Individual(tour=solution, fitness=fitness(solution, distance_matrix))


def generate_random_solution(n_cities: int) -> List[int]:
    """generate a random permutation of cities using numpy"""
    return np.random.permutation(n_cities).tolist()


def generate_population(
    pop_size: int, n_cities: int, dist: np.ndarray
) -> List[Individual]:
    """generate initial random population sorted by fitness"""
    population = [
        make_individual(generate_random_solution(n_cities), dist)
        for _ in range(pop_size)
    ]
    population.sort(key=lambda x: x.fitness)
    return population


# --- PARENT SELECTION ---
def tournament_selection(population: List[Individual], k: int = 3) -> Individual:
    """
    k controls selective pressure:
    - high k -> the real fittest more likely to win -> more selective pressure (converges faster)
    - low k -> more chances to win for anyone -> less selective pressure, more random (... slower)
    the idea is to increase k for smaller instances of the tsp problem and decrease for bigger ones
    """
    competitors = random.sample(population, k)
    winner = min(competitors, key=lambda x: x.fitness)
    return winner


# alternative
def rank_selection(population: List[Individual], p: float = 1.0) -> Individual:
    """
    rank-based selection tweaking selective pressure with p (requires initial sorting)
    - p = 0  -> uniform random (each individual has the same probability to be selected)
    - p = 1  -> linear ranking
    - p > 1  -> exponential ranking (more selective pressure, more chances for the fittest)
    """
    N = len(population)
    ranks = np.arange(N, 0, -1)  # [N, N-1, ..., 1]
    weights = ranks**p  # (r_i ^ p)
    probs = weights / weights.sum()  # normalize weights to probs
    idx = np.random.choice(N, p=probs)
    return population[idx]  # return parent


# --- debug ---
# n_cities = len(CITIES)
# pop = generate_population(pop_size=10, n_cities=n_cities, distance_matrix=test_problem)
# ic(pop)


In [52]:
# --- MUTATION FUNCTIONS ---
def mutate_swap(solution: List[int]) -> List[int]:
    """swap two random cities"""
    sol = solution.copy()
    i, j = np.random.choice(len(sol), 2, replace=False)
    sol[i], sol[j] = sol[j], sol[i]
    return sol


def mutate_insertion(solution: List[int]) -> List[int]:
    """remove a random city and reinsert it in another random position"""
    sol = solution.copy()
    i, j = np.random.choice(len(sol), 2, replace=False)
    city = sol.pop(i)
    sol.insert(j, city)
    return sol


def mutate_inversion(solution: List[int]) -> List[int]:
    """reverse a random segment (should be more efficient for instances with symmetrical matrix)"""
    sol = solution.copy()
    i, j = sorted(np.random.choice(len(sol), 2, replace=False))
    sol[i : j + 1] = reversed(sol[i : j + 1])
    return sol


# test
def random_mutation(sol: List[int]) -> List[int]:
    ops = [mutate_swap, mutate_insertion, mutate_inversion]
    op = random.choice(ops)
    return op(sol)


# --- RECOMBINATION/CROSSOVER FUNCTIONS ---
def order_crossover(p1, p2):
    """copy a random slice from p1 then fill the remaining positions from p2 in order"""
    size = len(p1)
    a, b = sorted(np.random.choice(size, 2, replace=False))
    child = [None] * size
    child[a:b] = p1[a:b]  # [a,b)
    fill = [c for c in p2 if c not in child]
    idx = [i for i in range(size) if child[i] is None]
    for i, c in zip(idx, fill):
        child[i] = c
    return child


def inver_over_v1(p1: List[int], p2: List[int]) -> List[int]:
    """steps:
    1. select a random starting city from p1
    2. find its successor in the guiding parent p2
    3. if the edge already exists in p1 -> stop
    4. otherwise, invert the segment between the two cities in p1
    (should work very well with symmetric tsp)
    """
    tour = p1.copy()
    n = len(tour)
    city = random.choice(tour)  # step 1: random starting city
    idx2 = p2.index(city)
    next_city = p2[(idx2 + 1) % n]  # step 2: find successor in guiding parent
    i, j = tour.index(city), tour.index(next_city)
    if (i + 1) % n == j or (j + 1) % n == i:
        return tour  # step 3: if they are already adjacent -> stop (nothing to do)
    a, b = sorted([i, j])
    tour[a + 1 : b + 1] = list(
        reversed(tour[a + 1 : b + 1])
    )  # step 5: otherwise invert the segment between them
    return tour


# just an alternative, inver-over used as single operator
def inver_over_v2(p1: List[int], p2: List[int], prob: float = 0.02) -> List[int]:
    tour = p1.copy()
    n = len(tour)
    city = np.random.choice(tour)
    while True:
        if np.random.random() < prob:  # select from p2 (like crossover)
            idx2 = p2.index(city)
            next_city = p2[(idx2 + 1) % n]
        else:  # select from p1 (more like mutation)
            next_city = np.random.choice([c for c in tour if c != city])
        i, j = tour.index(city), tour.index(next_city)
        if (i + 1) % n == j or (j + 1) % n == i:
            break  # stop condition
        a, b = sorted([i, j])
        tour[a + 1 : b + 1] = list(reversed(tour[a + 1 : b + 1]))
        city = next_city
    return tour


def opt2_localsearch(
    tour: list[int], dist: np.ndarray, k: int = 8, max_passes: int = 5
) -> list[int]:
    """additional local search for improving child after mutation/crossover"""
    best = np.array(tour, dtype=int)
    n = len(best)
    # precompute neighbors (for faster computation)
    # k is the number of neigbours
    neighbors = np.argsort(dist, axis=1)[:, 1 : k + 1]
    # map city -> position in tour
    position = np.empty(n, dtype=int)
    for i, city in enumerate(best):
        position[city] = i
    passes = 0  # attempts without improvement
    improved = True
    while improved and passes < max_passes:
        improved = False
        for i in range(n - 2):
            a, b = best[i], best[(i + 1) % n]  # first edge to cut (a,b)
            dab = dist[a, b]
            for c in neighbors[a]:
                j = position[c]
                if j <= i + 1 or (j == n - 1 and i == 0):
                    continue
                d = best[(j + 1) % n]  # second edge to cut (c,d)
                # now compute delta cutting (a,b), (c,d) and making new edges (a,c), (b,d)
                delta = (dist[a, c] + dist[b, d]) - (dab + dist[c, d])
                if delta < -1e-12:  # improvemente (delta < 0)
                    left, right = i + 1, j
                    while left < right:
                        best[left], best[right] = best[right], best[left]
                        position[best[left]] = left
                        position[best[right]] = right
                        left += 1
                        right -= 1
                    improved = True
                    break
            if improved:
                break
        passes += 1  # if no improvement
    return best.tolist()


In [53]:
def steady_state(
    population, distance_matrix, POP_SIZE: int, generations: int, k: int, rate: float
):
    """steady-state approach (replaces the worst individual only if offspring is better)"""

    population.sort(key=lambda x: x.fitness)
    for step in range(generations):
        # --- generate offspring ---
        if np.random.random() < rate:
            p = tournament_selection(population, k)
            child = mutate_swap(p.tour)
            # child = mutate_insertion(p.tour)
            # child = mutate_inversion(p.tour)
            # child = random_mutation(p.tour)
        else:
            p1, p2 = (
                tournament_selection(population, k),
                tournament_selection(population, k),
            )
            # child = inver_over_v1(p1.tour, p2.tour)
            child = order_crossover(p1.tour, p2.tour)

        child = make_individual(child, distance_matrix)
        # --- replace only if better than worst ---
        if child.fitness < population[-1].fitness:
            population[-1] = child
            population.sort(key=lambda x: x.fitness)  # keep sorted

        if step % 1000 == 0:
            print(f"Step {step:5d} | best = {population[0].fitness:.2f}")
    return population[0]  # best individual

In [54]:
def generational(
    population,
    distance_matrix,
    POP_SIZE: int,
    rate: float,
    elitism: int,
    generations: int,
    k: int = 3,
):
    """survival selection (generational approach + some elitism)"""

    prev_best = population[0].fitness
    stagnation = 0
    # the population was previously sorted
    for gen in range(generations):
        elites = population[:elitism]  # some elitism (preserve best)
        new_population = elites.copy()
        while len(new_population) < POP_SIZE:
            if np.random.random() < rate:
                p = tournament_selection(population, k)
                child = mutate_swap(p.tour)
                # child = mutate_insertion(p.tour)
                # child = mutate_inversion(p.tour)
                # child = random_mutation(p.tour)
            else:
                p1, p2 = (
                    tournament_selection(population, k),
                    tournament_selection(population, k),
                )
                # child = inver_over_v1(p1.tour, p2.tour)
                child = order_crossover(p1.tour, p2.tour)
            new_population.append(make_individual(child, distance_matrix))
            # the population is entirely replaced (except for elitism)
        new_population.sort(key=lambda x: x.fitness)
        population = new_population[:POP_SIZE]

        if gen % 100 == 0:
            print(f"Gen {gen:5d} | best = {population[0].fitness:.2f}")

        best = population[0].fitness
        if abs(prev_best - best) < 1e-6:  # no significant difference
            stagnation += 1
        else:
            stagnation = 0
        prev_best = best

        if stagnation > 500:
            print(f"Converged early at generation {gen}")
            break

    return min(population, key=lambda x: x.fitness)

In [60]:
def hybrid(
    population,
    distance_matrix,
    POP_SIZE: int,
    generations: int,
    k: int,
    elitism: int,
    rate: float,
    OFFSPRING_SIZE: int,
):
    """first generate all the OFFSPRINGS, then add it to the population and they compete for survival"""
    prev_best = population[0].fitness
    stagnation = 0
    # the population was previously sorted
    for gen in range(generations):
        elites = population[:elitism]  # some elitism
        # --- generate λ offspring ---
        λ = []
        while len(λ) < OFFSPRING_SIZE:
            if np.random.random() < rate:
                p = tournament_selection(population, k)
                child = mutate_swap(p.tour)
                # child = mutate_insertion(p.tour)
                # child = mutate_inversion(p.tour)
                # child = random_mutation(p.tour)
            else:
                p1, p2 = (
                    tournament_selection(population, k),
                    tournament_selection(population, k),
                )
                # child = inver_over_v1(p1.tour, p2.tour)
                child = order_crossover(p1.tour, p2.tour)
            λ.append(make_individual(child, distance_matrix))

        # --- merge λ and elites into the population ---
        population.extend(λ)
        population.extend([e for e in elites if e not in population])
        population.sort(key=lambda x: x.fitness)
        population = population[:POP_SIZE]  # sort and keep best individuals

        if gen % 100 == 0:
            print(f"Gen {gen:5d} | best = {population[0].fitness:.2f}")

        best = population[0].fitness
        if abs(prev_best - best) < 1e-6:  # no significant difference
            stagnation += 1
        else:
            stagnation = 0
        prev_best = best

        if stagnation > 500:
            print(f"Converged early at generation {gen}")
            break

    return min(population, key=lambda x: x.fitness)

In [65]:
def hybrid_v2(
    population,
    distance_matrix,
    POP_SIZE: int,
    generations: int,
    k: int,
    elitism: int,
    mutation_rate: float,
    OFFSPRING_SIZE: int,
    local_search_rate: float = 0.1,
):
    """the idea here is to improve the first hybrid model with some additional local search:
    - NOT after every mutation/crossover (really too slow for large instances) but only for 10% offsprings
    - or use it on the best few individuals every N generations (e.g. every 100)
    - or trigger it at the end, after stagnation (i.e. no improvement for many gens)
    maybe not in early generations otherwise the model converges to a local optima too fast
    this keeps diversity early and refines tours later when population stabilizes
    """
    prev_best = population[0].fitness
    stagnation = 0
    for gen in range(generations):
        elites = population[:elitism]
        offspring = []
        # --- generate λ offspring ---
        while len(offspring) < OFFSPRING_SIZE:
            p1 = tournament_selection(population, k)
            p2 = tournament_selection(population, k)
            # child_tour = inver_over_v1(p1.tour, p2.tour)
            child_tour = order_crossover(p1.tour, p2.tour)
            # additional local search (! this is really slow)
            if np.random.random() < local_search_rate:  # only 10% offsprings
                child_tour = opt2_localsearch(child_tour, distance_matrix)

            if np.random.random() < mutation_rate:
                if np.random.random() < 0.5:
                    child_tour = mutate_swap(child_tour)
                else:
                    child_tour = mutate_inversion(child_tour)

            offspring.append(make_individual(child_tour, distance_matrix))

        # --- combine and sort population ---
        population.extend(offspring)
        population.extend(elites)
        population.sort(key=lambda x: x.fitness)
        population = population[:POP_SIZE]

        if gen % 100 == 0:
            """ 
            # late-stage 2-opt refinement only on best individuals 
            if gen % 500 == 0 and gen > generations * 0.3:  # after 30% of run (not in early generations)
                print(f"Refining with 2-opt at gen {gen}")
                top_k = min(3, len(population))
                for i in range(top_k):
                    improved_tour = opt2_localsearch(population[i].tour, distance_matrix)
                    population[i] = make_individual(improved_tour, distance_matrix)
                # re-sort after refinement
                population.sort(key=lambda x: x.fitness)
            """
            print(
                f"Gen {gen:5d} | best = {population[0].fitness:.2f} | mut={mutation_rate:.2f} | k={k}"
            )

        best_fit = population[0].fitness
        if abs(prev_best - best_fit) < 1e-6:
            stagnation += 1
        else:
            stagnation = 0
        prev_best = best_fit

        # --- dynamic parameter adaptation ---
        if gen % 100 == 0 and gen > 0:
            if stagnation > 50:
                # stagnation -> encourage diversity
                mutation_rate = max(mutation_rate * 0.9, 0.5)
                k = max(2, k - 1)
            else:
                # improving -> more exploitation
                mutation_rate = min(mutation_rate * 1.1, 0.8)
                k = min(k + 1, 5)

        # if stagnation > 1000:
        #     print(f"Refining with 2-opt after stagnation")
        #     top_k = min(10, len(population))
        #     for i in range(top_k):
        #         improved_tour = opt2_localsearch(population[i].tour, distance_matrix)
        #         population[i] = make_individual(improved_tour, distance_matrix)
        #     # re-sort after refinement
        #     population.sort(key=lambda x: x.fitness)
        #     print(f"Converged early at generation {gen}")
        #     break

    return population[0]

In [57]:
# just a test
def pure_inver_over(
    population, distance_matrix, generations: int = 1000, k: int = 3, prob: float = 0.02
):
    """pure inver-over as single operator"""
    n = len(population[0].tour)
    for gen in range(generations):
        new_pop = []
        for ind in population:
            p2 = tournament_selection(population, k)
            child_tour = inver_over_v2(ind.tour, p2.tour, prob)
            new_pop.append(make_individual(child_tour, distance_matrix))

        population.extend(new_pop)
        population.sort(key=lambda x: x.fitness)
        population = population[: len(new_pop)]  # to keep pop size fixed

        if gen % 100 == 0:
            print(f"Gen {gen:4d} | best = {population[0].fitness:.2f}")
    return population[0]

In [62]:
def run_tsp(
    distance_matrix,
    model: str = "hybrid",
    pop_size: int = 100,
    generations: int = 1000,
    k: int = 3,
    elitism: int = 5,
    rate: float = 0.3,
    offspring_size: int = 20,
):
    """
    run a TSP evolutionary algorithm using the specified model
    where model ∈ {'steady', 'generational', 'hybrid', 'inver-over'}
    """
    n_cities = len(distance_matrix)
    population = generate_population(pop_size, n_cities, distance_matrix)
    print(f"Initial best = {population[0].fitness:.2f}")

    if model == "steady":
        best = steady_state(population, distance_matrix, pop_size, generations, k, rate)
    elif model == "generational":
        best = generational(
            population, distance_matrix, pop_size, rate, elitism, generations, k
        )
    elif model == "hybrid":
        best = hybrid_v2(
            population,
            distance_matrix,
            pop_size,
            generations,
            k,
            elitism,
            rate,
            offspring_size,
        )
    elif model == "inver-over":
        best = pure_inver_over(population, distance_matrix, generations, k)
    else:
        raise ValueError("model name not correct'")

    print(f"\nFinal best fitness: {best.fitness:.2f}")
    return best

In [66]:
best = run_tsp(
    problem,
    model="hybrid",  # 'steady', 'generational', or 'hybrid'
    pop_size=150,
    generations=30_000,
    elitism=3,
    k=3,
    rate=0.2,
    offspring_size=75,
)

print("Best tour:", best.tour)

Initial best = -735.01
Gen     0 | best = -735.01 | mut=0.20 | k=3
Gen   100 | best = -3237.69 | mut=0.20 | k=3
Gen   200 | best = -3486.57 | mut=0.22 | k=4
Gen   300 | best = -3717.11 | mut=0.24 | k=5
Gen   400 | best = -3825.04 | mut=0.27 | k=5
Gen   500 | best = -3879.47 | mut=0.29 | k=5
Gen   600 | best = -3943.43 | mut=0.50 | k=4
Gen   700 | best = -3968.29 | mut=0.55 | k=5
Gen   800 | best = -4017.41 | mut=0.61 | k=5
Gen   900 | best = -4036.06 | mut=0.67 | k=5
Gen  1000 | best = -4071.31 | mut=0.60 | k=4
Gen  1100 | best = -4071.31 | mut=0.54 | k=3
Gen  1200 | best = -4071.31 | mut=0.50 | k=2
Gen  1300 | best = -4071.31 | mut=0.50 | k=2
Gen  1400 | best = -4071.31 | mut=0.50 | k=2
Gen  1500 | best = -4086.97 | mut=0.50 | k=2
Gen  1600 | best = -4114.79 | mut=0.55 | k=3
Gen  1700 | best = -4114.79 | mut=0.50 | k=2
Gen  1800 | best = -4114.79 | mut=0.50 | k=2
Gen  1900 | best = -4114.79 | mut=0.50 | k=2
Gen  2000 | best = -4146.60 | mut=0.50 | k=2
Gen  2100 | best = -4158.81 | mut

KeyboardInterrupt: 